In [ ]:
import torch.nn as nn
from torch.nn import functional as F 
  
class IndConv2(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, mode='nearest'):
        super(IndConv2, self).__init__()
        
        self.mode = mode
        self.in_channels = in_channels
        self.A00 = nn.Conv2d(in_channels, out_channels, (3,3))
        self.A01 = nn.Conv2d(in_channels, out_channels, (3,3))
        self.A02 = nn.Conv2d(in_channels, out_channels, (3,3))
        self.A10 = nn.Conv2d(in_channels, out_channels, (3,3))
        self.A11 = nn.Conv2d(in_channels, out_channels, (3,3))
        self.A12 = nn.Conv2d(in_channels, out_channels, (3,3))
        self.A20 = nn.Conv2d(in_channels, out_channels, (3,3))
        self.A21 = nn.Conv2d(in_channels, out_channels, (3,3))
        self.A22 = nn.Conv2d(in_channels, out_channels, (3,3))
    
    def forward(self, x):
        ins = F.interpolate(x.reshape(x.shape[0], self.in_channels, 3, 3), scale_factor=3), mode=self.mode)
        out = torch.ones((x.shape[0], self.in_channels, 3, 3)).type(torch.cuda.FloatTensor)
    
        out[:, :, 0, 0] = self.A00(ins[:, :, 2:5, 2:5])[:, :, 0, 0]
        out[:, :, 0, 2] = self.A02(ins[:, :, 2:5, 4:7])[:, :, 0, 0]
        out[:, :, 2, 2] = self.A22(ins[:, :, 4:7, 4:7])[:, :, 0, 0]
        out[:, :, 2, 0] = self.A20(ins[:, :, 4:7, 2:5])[:, :, 0, 0]
        out[:, :, 1, 0] = self.A10(ins[:, :, 3:6, 2:5])[:, :, 0, 0]
        out[:, :, 0, 1] = self.A01(ins[:, :, 2:5, 3:6])[:, :, 0, 0]
        out[:, :, 1, 2] = self.A12(ins[:, :, 3:6, 4:7])[:, :, 0, 0]
        out[:, :, 2, 1] = self.A21(ins[:, :, 4:7, 3:6])[:, :, 0, 0]
        out[:, :, 1, 1] = self.A11(ins[:, :, 3:6, 3:6])[:, :, 0, 0]

        return out